In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader , random_split
from torchvision import datasets, transforms, models

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [3]:
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])


transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

full_train_dataset = datasets.ImageFolder(root='dataset/train', transform=transform_train)

train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

test_dataset = datasets.ImageFolder(root='dataset/test', transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

num_classes = len(full_train_dataset.classes)
print(f"Number of classes: {num_classes}")


Number of classes: 5


In [4]:
model = models.resnet50(pretrained=True)

for name,params in model.named_parameters():
    if 'layer4' in name or 'layer3' in name or 'fc' in name:
        params.requires_grad = True
    else:
        params.requires_grad = False

num_filters = model.fc.in_features
model.fc = nn.Linear(num_filters,num_classes)

model = model.to(device)

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p : p.requires_grad, model.parameters()), lr=0.00001)  # only train fc layer

In [6]:
epochs = 10  # adjust as needed

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_acc = 100 * val_correct / val_total

    print(f"Epoch [{epoch+1}/{epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")


Epoch [1/10] Train Loss: 1.4374, Train Acc: 41.26% | Val Loss: 1.2071, Val Acc: 64.55%
Epoch [2/10] Train Loss: 1.0432, Train Acc: 75.23% | Val Loss: 0.9006, Val Acc: 79.82%
Epoch [3/10] Train Loss: 0.7712, Train Acc: 86.93% | Val Loss: 0.6573, Val Acc: 83.64%
Epoch [4/10] Train Loss: 0.5602, Train Acc: 89.89% | Val Loss: 0.5121, Val Acc: 88.18%
Epoch [5/10] Train Loss: 0.4336, Train Acc: 91.48% | Val Loss: 0.4189, Val Acc: 87.82%
Epoch [6/10] Train Loss: 0.3335, Train Acc: 92.12% | Val Loss: 0.3489, Val Acc: 90.55%
Epoch [7/10] Train Loss: 0.2846, Train Acc: 92.85% | Val Loss: 0.3195, Val Acc: 90.55%
Epoch [8/10] Train Loss: 0.2437, Train Acc: 94.22% | Val Loss: 0.2928, Val Acc: 90.55%
Epoch [9/10] Train Loss: 0.2171, Train Acc: 94.49% | Val Loss: 0.2604, Val Acc: 91.45%
Epoch [10/10] Train Loss: 0.1767, Train Acc: 95.90% | Val Loss: 0.2570, Val Acc: 90.91%


In [7]:
model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100*test_correct/test_total:.2f}%")


Test Accuracy: 91.69%


In [8]:
torch.save(model.state_dict(), "flower_detector.pth")
print("Model saved successfully!")


Model saved successfully!


In [9]:
from PIL import Image

def predict_image(image_path):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        _, pred = torch.max(output, 1)
    return full_train_dataset.classes[pred.item()]

In [16]:
print(predict_image('dataset/test/Image_844.jpg'))

tulip
